In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns
from datetime import datetime, timedelta
from scipy.stats import pearsonr
from scipy.stats import kendalltau
from scipy.stats import spearmanr

In [2]:
df_transactions_wofirstname = pd.read_csv("C:/Users/felix.oechslein/Desktop/Daten, 13.12.22/Rohdaten/" +
                                    "Thalia_transactions_ohne_vorname.csv")

df_history_wofirstname = pd.read_csv("C:/Users/felix.oechslein/Desktop/Daten, 13.12.22/Rohdaten/" +
                                    "Thalia_history_ohne_vorname.csv")

## 0. Daten betrachten und "df_test" als Test DataFrame

In [3]:
df_history_wofirstname.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4854430 entries, 0 to 4854429
Data columns (total 7 columns):
 #   Column    Dtype 
---  ------    ----- 
 0   mandator  int64 
 1   id        int64 
 2   iid       int64 
 3   attrtype  object
 4   akey      int64 
 5   text      object
 6   creation  int64 
dtypes: int64(5), object(2)
memory usage: 259.3+ MB


In [4]:
df_test = df_history_wofirstname["text"]

In [5]:
df_test.head(10)

0    |msg=Einladung_Fraternitee_Lounge|userAgent=Ap...
1    |msg=Einladung_Fraternitee_Lounge|userAgent=Mo...
2    |msg=NL_April_1_2022|userAgent=Thunderbird 91....
3    |msg=Einladung_Fraternitee_Lounge|userAgent=Mo...
4    |msg=NL_März_2_2022|userAgent=Apple Mail (OS X...
5    |msg=NL_Dezember_1_2022|hidden=true|fullUserAg...
6    |msg=NL_November_2_2022|hidden=true|fullUserAg...
7    |msg=Save the Date_Fraternite_Tartuffe|userAge...
8    |msg=SM: Network 31.03.2022|userAgent=Mobile S...
9    |msg=NL_April_1_2022|userAgent=unknown (unknow...
Name: text, dtype: object

In [6]:
df_test.info()

<class 'pandas.core.series.Series'>
RangeIndex: 4854430 entries, 0 to 4854429
Series name: text
Non-Null Count    Dtype 
--------------    ----- 
4854430 non-null  object
dtypes: object(1)
memory usage: 37.0+ MB


## 1. Erstellung des "df_neueSpalte" für die Betrachtung sämtlicher Infos zwischen den "|"

In [7]:
neueSpalten = [ line.strip().split('|') for line in df_test ]

In [8]:
df_neueSpalte = pd.DataFrame(neueSpalten)
df_neueSpalte.head(5)

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,,msg=Einladung_Fraternitee_Lounge,userAgent=Apple Mail (OS X / Personal computer),hidden=true,ip=192.119.58.218,userLanguage=de,,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,,msg=Einladung_Fraternitee_Lounge,userAgent=Mobile Safari (iOS / Smartphone),hidden=true,ip=80.133.234.253,userLanguage=de,,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,,msg=NL_April_1_2022,userAgent=Thunderbird 91.7 (Windows 8.1 / Pers...,hidden=true,ip=95.33.57.78,userLanguage=de,,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,,msg=Einladung_Fraternitee_Lounge,userAgent=Mobile Safari (iOS / Smartphone),hidden=true,ip=80.133.234.253,userLanguage=de,,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,,msg=NL_März_2_2022,userAgent=Apple Mail (OS X / Personal computer),hidden=true,ip=79.197.199.159,userLanguage=de,,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## 2. Ein neues DataFrame df_user_agent und eine neue Spalte "User agent", die noch sämtliche Informationen danach enthält

In [71]:
df_user_agent = pd.DataFrame()
df_user_agent["User Agent"] = df_test.str.extract(r'userAgent=(.*)', expand=False)

In [72]:
df_user_agent.head()

,User Agent
0,Apple Mail (OS X / Personal computer)|hidden=t...
1,Mobile Safari (iOS / Smartphone)|hidden=true|i...
2,Thunderbird 91.7 (Windows 8.1 / Personal compu...
3,Mobile Safari (iOS / Smartphone)|hidden=true|i...
4,Apple Mail (OS X / Personal computer)|hidden=t...


## 3. Reduktion auf "User Agent" Daten

In [73]:
df_user_agent["User Agent"] = df_user_agent["User Agent"].apply(lambda zeile: zeile.split("|")[0] if isinstance(zeile, str) else zeile)

In [74]:
df_user_agent.head()

,User Agent
0,Apple Mail (OS X / Personal computer)
1,Mobile Safari (iOS / Smartphone)
2,Thunderbird 91.7 (Windows 8.1 / Personal compu...
3,Mobile Safari (iOS / Smartphone)
4,Apple Mail (OS X / Personal computer)


## 4. Ergänzung duch Infos für welche Datenpunkte Probleme bei der Genauigkeit der Öffnungsrate besteht

In [75]:
df_user_agent_aufgeteilt = df_user_agent

# Spalten mit Ja/Nein-Werten erstellen
df_user_agent_aufgeteilt["unknown"] = df_user_agent['User Agent'].str.contains('unknown', case=False)
#df_user_agent_aufgeteilt['iOS-Nutzer'] = df_user_agent['User Agent'].str.contains('iOS', case=False)
#df_user_agent_aufgeteilt['OS-X-Nutzer'] = df_user_agent['User Agent'].str.contains('OS X|Macintosh', case=False)
df_user_agent_aufgeteilt['Apple-Mail-Nutzer'] = df_user_agent['User Agent'].str.contains('Apple Mail', case=False)
df_user_agent_aufgeteilt['Gmail-Nutzer'] = df_user_agent['User Agent'].str.contains('Gmail', case=False)
df_user_agent_aufgeteilt['Proxy-Server'] = df_user_agent['User Agent'].str.contains('Proxy', case=False)
df_user_agent_aufgeteilt.head()

,User Agent,unknown,Apple-Mail-Nutzer,Gmail-Nutzer,Proxy-Server
0,Apple Mail (OS X / Personal computer),False,True,False,False
1,Mobile Safari (iOS / Smartphone),False,False,False,False
2,Thunderbird 91.7 (Windows 8.1 / Personal compu...,False,False,False,False
3,Mobile Safari (iOS / Smartphone),False,False,False,False
4,Apple Mail (OS X / Personal computer),False,True,False,False


## 5. Berechnung wie viel Prozent der Datenpunkte nicht für die Berechnung der Öffnungsrate verwendet werden können

## -> rund 40%

In [76]:
for col in df_user_agent.columns[1:]:
    print(df_user_agent[col].value_counts())

False    1743162
True      821182
Name: unknown, dtype: int64
False    2131020
True      433324
Name: Apple-Mail-Nutzer, dtype: int64
False    2434587
True      129757
Name: Gmail-Nutzer, dtype: int64
False    2434587
True      129757
Name: Proxy-Server, dtype: int64


In [77]:
anzahl = (df_user_agent.iloc[:, 1:] == True).any(axis=1).sum()
print(f"Anzahl der Zeilen mit mindestens einem True (ohne erste Spalte): {anzahl}")
gesamtzahl = df_test.size
print(f"Gesamtzahl der Zeilen: {gesamtzahl}")
prozentsatz = 1 - anzahl / gesamtzahl
print(f"Prozentsatz verwendeter Datenpunkte: {prozentsatz}")

Anzahl der Zeilen mit mindestens einem True (ohne erste Spalte): 1384263
Gesamtzahl der Zeilen: 4854430
Prozentsatz verwendeter Datenpunkte: 0.7148454092447517


# Tryouts

In [39]:
df_neueSpalte["User Agent"].nunique()

1787

In [70]:
df_neueSpalte["User Agent"].value_counts().head(50)

                                                        2290086
unknown (unknown / Personal computer)                    797754
Apple Mail (OS X / Personal computer)                    416920
Mobile Safari (iOS / Smartphone)                         370916
Gmail image proxy (Linux / Other)                        106224
Mobile Safari (iOS / Tablet)                              34535
Android browser 4.0 (Android / Smartphone)                24430
Gmail image proxy (Windows / Other)                       23533
Chrome Mobile 92.0 (Android / Smartphone)                 18197
Chrome Mobile 83.0 (Android / Smartphone)                 17822
Chrome Mobile 107.0 (Android / Smartphone)                17100
Chrome Mobile 106.0 (Android / Smartphone)                17026
Chrome Mobile 96.0 (Android / Smartphone)                 16784
Outlook 16 (unknown / Personal computer)                  15455
Mobile Safari 14.1 (iOS / Smartphone)                     12898
Chrome Mobile 102.0 (Android / Smartphon